# LangChain MCP Adapters - Learning Notebook

This notebook demonstrates how to integrate LangChain agents with MCP (Model Context Protocol) servers.

## Learning Path
1. Single MCP server connection (stdio transport)
2. Multiple MCP servers (stdio + streamable-http transports)
3. Streamable HTTP transport
4. Passing runtime headers
5. Using with LangGraph StateGraph
6. Converting LangChain tools to MCP format

## Prerequisites
- Start the weather server: `python servers/weather_server.py`
- Start the langchain tools server: `python servers/langchain_tools_server.py --port 8001`

## Libraries

In [60]:
from typing import Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from display_utils import display_agent_response, get_final_answer, print_tools_summary

### Client

In [61]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["/home/donbr/don-aie-cohort8/aie8-s13-langchain-mcp/servers/math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent("openai:gpt-4.1", tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [62]:
agent_response = await agent.ainvoke({"messages": [HumanMessage("what's (3 + 5) x 12?")]})

In [63]:
# Full trace
display_agent_response(agent_response)


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ❌ Error: ClosedResourceError()
 Please fix your mistakes.
04. AIMessage → 🔧 tool_call(s): add
05. ToolMessage [add]: ❌ Error: ClosedResourceError()
 Please fix your mistakes.
06. AIMessage: To solve (3 + 5) × 12:
First, calculate 3 + 5 = 8.
Then, multiply 8 × 12 = 96.

So, (3 + 5) x 12 = 96.




### Understanding ClosedResourceError

**What you're seeing:** `ClosedResourceError()` in the tool responses above.

**Why it happens:** The MCP session is closed after the initial tool discovery phase because we used an `async with` context manager. When the agent tries to call the tools, the session is no longer available.

**The fix:** Use `MultiServerMCPClient` (shown in later examples) which manages session lifecycle properly, keeping connections open for tool invocations.

**Key lesson:** When using stdio transport with context managers, the session must remain open during agent execution. The `MultiServerMCPClient` pattern handles this correctly.

In [64]:
# Just the answer
display_agent_response(agent_response, show_full_trace=False)


💡 Final Answer: To solve (3 + 5) × 12:
First, calculate 3 + 5 = 8.
Then, multiply 8 × 12 = 96.

So, (3 + 5) x 12 = 96.



In [65]:
# Get answer as variable
answer = display_agent_response(agent_response, return_final_answer=True)
print(f"The answer is: {answer}")


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ❌ Error: ClosedResourceError()
 Please fix your mistakes.
04. AIMessage → 🔧 tool_call(s): add
05. ToolMessage [add]: ❌ Error: ClosedResourceError()
 Please fix your mistakes.
06. AIMessage: To solve (3 + 5) × 12:
First, calculate 3 + 5 = 8.
Then, multiply 8 × 12 = 96.

So, (3 + 5) x 12 = 96.


The answer is: To solve (3 + 5) × 12:
First, calculate 3 + 5 = 8.
Then, multiply 8 × 12 = 96.

So, (3 + 5) x 12 = 96.


In [66]:
# Print tools summary
print_tools_summary(tools)


AVAILABLE TOOLS (2 total)

01. add
    └─ Add two numbers
02. multiply
    └─ Multiply two numbers




## Multiple MCP Servers

The library also allows you to connect to multiple MCP servers and load tools from them:

- math_server.py
- weather_server.py

### IMPORTANT: RUN WEATHER SERVER IN A SEPARATE TERMINAL

```bash
python weather_server.py
```

### Client

In [67]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/home/donbr/don-aie-cohort8/aie8-s13-langchain-mcp/servers/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # Make sure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

In [68]:
# Full trace
display_agent_response(math_response)


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add, multiply
03. ToolMessage [add]: ✓ 8
04. ToolMessage [multiply]: ✓ 96
05. AIMessage: (3 + 5) = 8. Then, 8 x 12 = 96. 

So, (3 + 5) x 12 = 96.




In [69]:
# Just the answer
display_agent_response(weather_response, show_full_trace=False)


💡 Final Answer: According to the latest available information, it's always sunny in New York City. If you want more detailed or current weather data, let me know!



In [70]:
# Get answer as variable
answer = display_agent_response(math_response, return_final_answer=True)
print(f"The answer is: {answer}")


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add, multiply
03. ToolMessage [add]: ✓ 8
04. ToolMessage [multiply]: ✓ 96
05. AIMessage: (3 + 5) = 8. Then, 8 x 12 = 96. 

So, (3 + 5) x 12 = 96.


The answer is: (3 + 5) = 8. Then, 8 x 12 = 96. 

So, (3 + 5) x 12 = 96.


> [!note]
> Example above will start a new MCP `ClientSession` for each tool invocation. If you would like to explicitly start a session for a given server, you can do:
>
> ```python
> from langchain_mcp_adapters.tools import load_mcp_tools
>
> client = MultiServerMCPClient({...})
> async with client.session("math") as session:
>     tools = await load_mcp_tools(session)
> ```

## Streamable HTTP

MCP now supports [streamable HTTP](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports#streamable-http) transport.

To start an [example](https://github.com/langchain-ai/langchain-mcp-adapters/tree/main/examples/servers/streamable-http-stateless) streamable HTTP server, run the following:

```bash
cd examples/servers/streamable-http-stateless/
uv run mcp-simple-streamablehttp-stateless --port 3000
```

Alternatively, you can use FastMCP directly (as in the examples above).

To use it with Python MCP SDK `streamablehttp_client`:

In [71]:
# Use server from examples/servers/streamable-http-stateless/

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

async with streamablehttp_client("http://localhost:3000/mcp") as (read, write, _):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        agent = create_react_agent("openai:gpt-4.1", tools)
        math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

Use it with `MultiServerMCPClient`:

In [43]:
# Use server from examples/servers/streamable-http-stateless/
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "math": {
            "transport": "streamable_http",
            "url": "http://localhost:3000/mcp"
        },
    }
)
tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [44]:
# Print the agent response
display_agent_response(math_response)


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ✓ 8
04. AIMessage → 🔧 tool_call(s): multiply
05. ToolMessage [multiply]: ✓ 96
06. AIMessage: (3 + 5) × 12 equals 96.




In [45]:
# Print the final answer
final_answer = get_final_answer(math_response)
print(f"The final answer is: {final_answer}")

The final answer is: (3 + 5) × 12 equals 96.


In [46]:
# Print the tools summary
print_tools_summary(tools)


AVAILABLE TOOLS (2 total)

01. add
    └─ Adds two numbers
02. multiply
    └─ Multiplies two numbers




## Passing runtime headers

When connecting to MCP servers, you can include custom headers (e.g., for authentication or tracing) using the `headers` field in the connection configuration. This is supported for the following transports:

- `sse`
- `streamable_http`

### Example: passing headers with `MultiServerMCPClient`

In [47]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "weather": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
            "headers": {
                "Authorization": "Bearer YOUR_TOKEN",
                "X-Custom-Header": "custom-value"
            },
        }
    }
)
tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

In [48]:
# Print the agent response
display_agent_response(response)


AGENT RESPONSE TRACE

01. HumanMessage: what is the weather in nyc?
02. AIMessage → 🔧 tool_call(s): get_weather
03. ToolMessage [get_weather]: ✓ It's always sunny in New York
04. AIMessage: The weather in New York City is described as always sunny. However, if you need the current detailed weather (such as temperature, humidity, or forecast), let me know!




In [49]:
# Print the final answer
final_answer = get_final_answer(response)
print(f"The final answer is: {final_answer}")

The final answer is: The weather in New York City is described as always sunny. However, if you need the current detailed weather (such as temperature, humidity, or forecast), let me know!


In [50]:
# Print the tools summary
print_tools_summary(tools)


AVAILABLE TOOLS (1 total)

01. get_weather
    └─ Get weather for location.




> Only `sse` and `streamable_http` transports support runtime headers. These headers are passed with every HTTP request to the MCP server.

## Using with LangGraph StateGraph

In [51]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition

from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-4.1")

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/home/donbr/don-aie-cohort8/aie8-s13-langchain-mcp/servers/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()

def call_model(state: MessagesState):
    response = model.bind_tools(tools).invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()
math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"})

In [52]:
# Print the math agent response
display_agent_response(math_response)


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ✓ 8
04. AIMessage → 🔧 tool_call(s): multiply
05. ToolMessage [multiply]: ✓ 96
06. AIMessage: (3 + 5) x 12 is 96.




In [53]:
# Print the weather agent response
display_agent_response(weather_response)


AGENT RESPONSE TRACE

01. HumanMessage: what is the weather in nyc?
02. AIMessage → 🔧 tool_call(s): get_weather
03. ToolMessage [get_weather]: ✓ It's always sunny in New York
04. AIMessage: According to my latest information, it's always sunny in New York City. If you need a more detailed or current weather update, please let me know!




In [54]:
# Print tools summary
print_tools_summary(tools)


AVAILABLE TOOLS (3 total)

01. add
    └─ Add two numbers
02. multiply
    └─ Multiply two numbers
03. get_weather
    └─ Get weather for location.




## Using with LangGraph API Server

In [72]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition

from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-4.1")

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/home/donbr/don-aie-cohort8/aie8-s13-langchain-mcp/servers/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()

def call_model(state: MessagesState):
    response = model.bind_tools(tools).invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()
math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"})

In [73]:
# print the math agent response
display_agent_response(math_response)


AGENT RESPONSE TRACE

01. HumanMessage: what's (3 + 5) x 12?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ✓ 8
04. AIMessage → 🔧 tool_call(s): multiply
05. ToolMessage [multiply]: ✓ 96
06. AIMessage: (3 + 5) is 8. When you multiply that by 12, you get 96. So, (3 + 5) x 12 = 96.




In [74]:
# print the weather agent response
display_agent_response(weather_response)


AGENT RESPONSE TRACE

01. HumanMessage: what is the weather in nyc?
02. AIMessage → 🔧 tool_call(s): get_weather
03. ToolMessage [get_weather]: ✓ It's always sunny in New York
04. AIMessage: The weather in New York City is described as always sunny. If you need current, detailed weather information (like temperature, humidity, or forecast), let me know!




## Using with LangGraph API Server

> [!TIP]
> Check out [this guide](https://langchain-ai.github.io/langgraph/tutorials/langgraph-platform/local-server/) on getting started with LangGraph API server.

If you want to run a LangGraph agent that uses MCP tools in a LangGraph API server, you can use the following setup:

In [56]:
# graph.py
from contextlib import asynccontextmanager
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

async def make_graph():
    client = MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                # Make sure to update to the full absolute path to your math_server.py file
                "args": ["/home/donbr/don-aie-cohort8/aie8-s13-langchain-mcp/servers/math_server.py"],
                "transport": "stdio",
            },
            "weather": {
                # make sure you start your weather server on port 8000
                "url": "http://localhost:8000/mcp",
                "transport": "streamable_http",
            }
        }
    )
    tools = await client.get_tools()
    agent = create_react_agent("openai:gpt-4.1", tools)
    return agent

In your [`langgraph.json`](https://langchain-ai.github.io/langgraph/cloud/reference/cli/#configuration-file) make sure to specify `make_graph` as your graph entrypoint:

```json
{
  "dependencies": ["."],
  "graphs": {
    "agent": "./graph.py:make_graph"
  }
}
```

## Add LangChain tools to a FastMCP server

Use `to_fastmcp` to convert LangChain tools to FastMCP, and then add them to the `FastMCP` server via the initializer:

> [!NOTE]
> `tools` argument is only available in FastMCP as of `mcp >= 1.9.1`

### RUN IN A SEPARATE TERMINAL

```bash
python langchain_tools_server.py --port 8001
```

In [57]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from display_utils import display_agent_response

client = MultiServerMCPClient({
    "langchain_math": {
        "url": "http://localhost:8001/mcp",
        "transport": "streamable_http",
    }
})

tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
response = await agent.ainvoke({"messages": "what is 15 + 27?"})

In [58]:
display_agent_response(response)


AGENT RESPONSE TRACE

01. HumanMessage: what is 15 + 27?
02. AIMessage → 🔧 tool_call(s): add
03. ToolMessage [add]: ✓ 42
04. AIMessage: 15 + 27 = 42.




In [59]:
# Print tools summary
print_tools_summary(tools)


AVAILABLE TOOLS (2 total)

01. add
    └─ Add two numbers
02. multiply
    └─ Multiply two numbers


